Attribution Policy:

    If using ACLED data in any way, direct or manipulated, the data must be clearly and prominently acknowledged. Proper acknowledgement includes (1) a footnote with the full citation which includes a/the link to ACLED’s website (see below for examples), (2) in-text citation/acknowledgement, stating that ACLED is the source of the data and that the data are publicly available, ​and​ (3) clear citation on any and all visuals making use of ACLED data.

    If generating a data file for public or private use, and presenting those data to another party, the ACLED data included must be directly acknowledged in a source column, including ACLED’s full name and a link: "Armed Conflict Location & Event Data Project (ACLED); www.acleddata.com​."
    To reference the ACLED codebook, please cite as follows (substituting for the correct year):

    "ACLED. (2019). "Armed Conflict Location & Event Data Project (ACLED) Codebook, 2019.""

    If using ACLED data in an academic paper or article, please cite as follows:

    "Raleigh, Clionadh, Andrew Linke, Håvard Hegre and Joakim Karlsen. (2010). “Introducing ACLED-Armed Conflict Location and Event Data.” Journal of Peace Research 47(5) 651-660."

Methodological sources used:
https://docs.bokeh.org/en/test/docs/user_guide/styling.html
https://discourse.bokeh.org/t/linking-daterangeslider-to-acled-data-on-bokeh-map/8848/2
https://stackoverflow.com/questions/46717651/bokeh-slider-using-pandas-datetime-index
https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-ii-interactions-a4cf994e2512
https://pbpython.com/pandas-qcut-cut.html
https://pbpython.com/natural-breaks.html

In [1]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import datetime
from datetime import date
from random import randint
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import pysal as ps

from bokeh.models import *
from bokeh.models.callbacks import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

import warnings

import os

warnings.filterwarnings("ignore")
# pd.set_option("max_columns", 30)

output_notebook()

Loading BokehJS ...

In [2]:
## DATA INDIVIDUAL CHILDREN ##
# path to individual_children transfer data
fp_individual_children = r'UKR_children/TEMPCopy_17112022_OTP2022 019586_children.xlsx'
# read individual_children transfer data into dataframe
individual_children = pd.read_excel(fp_individual_children, sheet_name='Children')

# create a 'count' column where each record = 1
individual_children['count'] = 1 

# rename some columns
individual_children = individual_children.rename(columns={'Surname and Given Names':'name', 
                                                          'Transferred/deported from':'from', 
                                                          'Transferred/deported to':'to',
                                                          'long_from':'lon_from','long_to':'lon_to',
                                                          'Date of transfer':'date_transfer'})

# clean up data
individual_children['lon_from'] = individual_children['lon_from'].astype(str).str.strip().str.rstrip(',').astype(float)
individual_children['lat_from'] = individual_children['lat_from'].astype(str).str.strip().str.rstrip(',').astype(float)
individual_children['lon_to'] = individual_children['lon_to'].astype(str).str.strip().str.rstrip(',').astype(float)
individual_children['lat_to'] = individual_children['lat_to'].astype(str).str.strip().str.rstrip(',').astype(float)
individual_children[['Age in 2022','from','to']] = individual_children[['Age in 2022','from','to']].replace(['','?','-','Ukraine'],'Unknown')
individual_children['date_transfer'] = individual_children['date_transfer'].replace(['?','-','Unknown'],'')
individual_children['date_transfer'] = individual_children['date_transfer'].replace(['In/prior to April 2022',
                                                                                     'At an unknown date – before 08/07/2022',
                                                                                     '02/03-19/05/2022'],
                                                                                    ['2022-04-01 00:00:00',
                                                                                     '2022-07-01 00:00:00',
                                                                                     '2022-03-02 00:00:00'])

# change dates to datetime format
individual_children['date_transfer'] = pd.to_datetime(individual_children['date_transfer'], errors='coerce')
# # create column with dates of transfer in dd-mm-yyyy string format
# individual_children['date_transfer_ddmmyyyy'] = individual_children['date_transfer'].dt.strftime('%d-%m-%Y')

# individual_children[['precision_from','precision_to']] = individual_children[['precision_from','precision_to']].astype(str)

# group the data per location of origin, destination and date + aggregate name children into a separate column
grouped_ind_child_name = individual_children.groupby(['date_transfer','from', 'to','precision_from','precision_to'], as_index=False).agg({'name' : ' '.join})
grouped_ind_child_count = individual_children.groupby(['date_transfer','from', 'to','precision_from','precision_to']).count()[['count']]
grouped_ind_child_lat_lon = individual_children.groupby(['date_transfer','from', 'to','precision_from','precision_to']).mean()[['lat_from', 'lon_from', 'lat_to', 'lon_to']]
grouped_ind_child_count = grouped_ind_child_count.join(grouped_ind_child_lat_lon).reset_index()
grouped_ind_child_count = grouped_ind_child_count.join(grouped_ind_child_name['name']).reset_index()
grouped_ind_child_count.drop('index', axis=1, inplace=True)
grouped_ind = grouped_ind_child_count

# creating bins
cut_bins = [0, 1, 20, 50, 100, 250, 500, 1000]
grouped_ind['bin_count_children'] = pd.cut(grouped_ind['count'], bins=cut_bins, labels=[2,5,8,15,25,40,60], include_lowest=True)

# transform lat and lon into Mercator coordinate system
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

cols = ['lon_to','lat_to','lon_from','lat_from','count']

lines_x, lines_y = [],[]
lons_to, lats_to, lons_from, lats_from = [], [], [], []
width = []

for lon_to, lat_to, lon_from, lat_from, count in grouped_ind[cols].values:
    lon_from, lat_from = transform(outProj,inProj,lon_from,lat_from)
    lon_to, lat_to = transform(outProj,inProj,lon_to,lat_to)

    lons_to.append(lon_to)
    lats_to.append(lat_to)
    
    lons_from.append(lon_from)
    lats_from.append(lat_from)

    lines_x.append([lon_from, lon_to])
    lines_y.append([lat_from, lat_to])
    
    width.append(count)
    
grouped_ind["mercator_lon_from"] = lons_from
grouped_ind["mercator_lat_from"] = lats_from
grouped_ind["mercator_lon_to"] = lons_to
grouped_ind["mercator_lat_to"] = lats_to 
grouped_ind["lines_x"] = lines_x
grouped_ind["lines_y"] = lines_y

# create column with dates of transfer in dd-mm-yyyy string format
grouped_ind['date_transfer_ddmmyyyy'] = grouped_ind['date_transfer'].dt.strftime('%d-%m-%Y')

# create colums to indicate data is about individual children for which name is known
grouped_ind['type'] = 'ID known'

# align columns with DATA GROUP CHILDREN
columns = ['count', 'type', 'date_transfer', 'from', 'precision_from', 'to', 'precision_to',
           'lat_from', 'lon_from', 'lat_to', 'lon_to',
           'bin_count_children', 'mercator_lon_from', 'mercator_lat_from',
           'mercator_lon_to', 'mercator_lat_to','lines_x','lines_y','date_transfer_ddmmyyyy']
grouped_ind = grouped_ind[columns]

# CDS data Container for Bokeh
# grouped_ind = grouped_ind.drop('geometry', axis=1)
# CDS_source_ind = ColumnDataSource(grouped_ind)

In [3]:
## DATA GROUP CHILDREN ##
# path to individual_children transfer data
fp_group_children = r'UKR_children/TEMPCopy_17112022_OTP2022 019586_children.xlsx'
# read individual_children transfer data into dataframe
group_children = pd.read_excel(fp_group_children, sheet_name='Locations_Children')

# rename some columns
group_children = group_children.rename(columns={'Number':'count',
                                                'Place from where deported':'from', 
                                                'Probable whereabouts':'to',
                                                'long_from':'lon_from','long_to':'lon_to',
                                                'Date of deportation':'date_transfer'})

# clean up data
group_children['lon_from'] = group_children['lon_from'].astype(str).str.strip().str.rstrip(',')
group_children['lat_from'] = group_children['lat_from'].astype(str).str.strip().str.rstrip(',')
group_children['lon_to'] = group_children['lon_to'].astype(str).str.strip().str.rstrip(',')
group_children['lat_to'] = group_children['lat_to'].astype(str).str.strip().str.rstrip(',')
group_children['date_transfer'] = group_children['date_transfer'].replace(['Approx. 06/2022',
                                                                           '14-15/07/2022',
                                                                           'Approx. 03/2022',
                                                                           'Approx. 15/07/2022',
                                                                           'Approx. 18/04/2022',
                                                                           'Approx. 23/05/2022',
                                                                           'Unknown'],
                                                                          ['2022-06-01 00:00:00',
                                                                           '2022-07-14 00:00:00',
                                                                           '2022-03-01 00:00:00',
                                                                           '2022-07-15 00:00:00',
                                                                           '2022-04-18 00:00:00',
                                                                           '2022-05-23 00:00:00',
                                                                           ''])

group_children['count'] = group_children['count'].replace(['13 children (10 to 17 years old)',
                                                           '135 children ',
                                                           '90 (10-15 years)',
                                                           'Group of children', # replace with smallest number = 2
                                                           '6 children (14 to 18 years old)',
                                                           '540 children',
                                                           '31 children',
                                                           '13+108+19 '], # 13+108+19=140
                                                          [13,
                                                           135,
                                                           90,
                                                           2,
                                                           6,
                                                           540,
                                                           31,
                                                           140])
                                                          
group_children['count'] = group_children['count'].astype(int)
                                                          
# change dates to datetime format
group_children['date_transfer'] = pd.to_datetime(group_children['date_transfer'], errors='coerce')
# create column with dates of transfer in dd-mm-yyyy string format
group_children['date_transfer_ddmmyyyy'] = group_children['date_transfer'].dt.strftime('%d-%m-%Y')

# creating bins
cut_bins = [0, 1, 20, 50, 100, 250, 500, 1000]
group_children['bin_count_children'] = pd.cut(group_children['count'], bins=cut_bins, labels=[2,5,8,15,25,40,60], include_lowest=True)

# transform lat and lon into Mercator coordinate system
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

cols = ['lon_to','lat_to','lon_from','lat_from','count']

lines_x, lines_y = [],[]
lons_to, lats_to, lons_from, lats_from = [], [], [], []
width = []

for lon_to, lat_to, lon_from, lat_from, count in group_children[cols].values:
    lon_from, lat_from = transform(outProj,inProj,lon_from,lat_from)
    lon_to, lat_to = transform(outProj,inProj,lon_to,lat_to)

    lons_to.append(lon_to)
    lats_to.append(lat_to)
    
    lons_from.append(lon_from)
    lats_from.append(lat_from)

    lines_x.append([lon_from, lon_to])
    lines_y.append([lat_from, lat_to])
    
    width.append(count)
    
group_children["mercator_lon_from"] = lons_from
group_children["mercator_lat_from"] = lats_from
group_children["mercator_lon_to"] = lons_to
group_children["mercator_lat_to"] = lats_to
group_children["lines_x"] = lines_x
group_children["lines_y"] = lines_y

# create colums to indicate data is about groups of children for which the identity is not known
group_children['type'] = 'ID unknown'

# align columns with DATA INDIVIDUAL CHILDREN
group_children = group_children[columns]

# CDS data Container for Bokeh
# group_children = group_children.drop('geometry', axis=1)
# CDS_source_grp = ColumnDataSource(group_children)

In [4]:
# group data on individual children for which the identity is known and groups of children for which the identity is not known
children_transferred = grouped_ind.append(group_children, ignore_index=True, verify_integrity=False, sort=None) 
# CDS data Container for Bokeh
CDS_source_children = ColumnDataSource(children_transferred)

In [47]:
children_transferred['lines_x'][24]
np.isnan(children_transferred['lines_x'][0]).any()

False

In [32]:
children_transferred['lines_x']

0      [3481862.0647935485, 3450197.781102907]
1     [4181605.2631029356, 4239669.7989313835]
2      [4181605.2631029356, 4153307.848543286]
3      [4177446.7899409644, 4153307.848543286]
4     [4177446.7899409644, 4153035.6501243976]
5      [4181605.2631029356, 4153307.848543286]
6     [4181605.2631029356, 4239669.7989313835]
7      [4181605.2631029356, 4629035.455047129]
8      [4181605.2631029356, 4629035.455047129]
9      [4181605.2631029356, 4153307.848543286]
10      [4153307.848543286, 4203535.469955988]
11      [4203535.469955988, 4332183.553811393]
12     [4181605.2631029356, 4113122.832371828]
13    [3936813.9811472544, 3928197.8302959567]
14    [4181605.2631029356, 4239669.7989313835]
15     [4203535.469955988, 4299637.2853499185]
16      [4037346.501963169, 4239417.237270672]
17     [4101010.0630880967, 4239417.237270672]
18     [4121771.2037807875, 4239417.237270672]
19     [3629972.8142732354, 4189199.790459427]
20     [3629972.8142732354, 4262529.946546625]
21     [36299

In [18]:
# https://stackoverflow.com/questions/41518920/python-pandas-how-to-query-if-a-list-type-column-contains-something

# df.genre.map(set(['comedy']).issubset)

# 0     True
# 1     True
# 2    False
# 3    False
# dtype: bool

children_transferred['lines_x'].map(set(['nan']).issubset)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
Name: lines_x, dtype: bool

In [24]:
def series_contains_nan(x):
    if x.values.any().isna():
        x = []
    else:
        x = x
    return x

children_transferred.loc[series_contains_nan(children_transferred['lines_x'])] = []

children_transferred['lines_x']

AttributeError: 'Series' object has no attribute 'series_contains_nan'

In [7]:
data = [1,np.nan]
test_series = pd.Series(data)
test_series

if test_series.isnull().values.any():
    test_series = []
print(test_series)

[]


In [8]:
def series_contains_nan(x):
    if x.values.any().isna():
        x = []
    else:
        x = x
    return x

# for idx, row in children_transferred.iterrows():
#     if series_contains_nan(row['lines_x']):
#         print(row['lines_x'])
    
#     break

# # children_transferred['lines_x']

In [9]:
## DATA ACLED ##

# path to ACLED conflict data
fp_acled = r'UKR_ACLED/CSV_2022-01-01-2022-11-08-Belarus-Moldova-Russia-Ukraine.csv'
# read ACLED conflict data into dataframe
df_acled = pd.read_csv(fp_acled)

# create a geodataframe out of 'conflict_data'
geo_acled = gpd.GeoDataFrame(df_acled, geometry=gpd.points_from_xy(df_acled.longitude, df_acled.latitude), crs=CRS.from_epsg(4326).to_wkt())

# Convert event date to pd.datetime format
geo_acled['event_date'] = pd.to_datetime(geo_acled['event_date'], errors='coerce')
geo_acled['event_date_ddmmyyyy'] = geo_acled['event_date'].dt.strftime('%d-%m-%Y')

# transform to Mercator CRS needed by Bokeh
geo_acled = geo_acled.to_crs(epsg=3857)

# Creating lat and lon in Mercator
geo_acled['mercator_lon'] = geo_acled['geometry'].x
geo_acled['mercator_lat'] = geo_acled['geometry'].y

# creating bins
cut_bins = [0, 1, 20, 50, 100, 250, 500]
geo_acled['bin_fatalities'] = pd.cut(geo_acled['fatalities'] , bins=cut_bins, labels=[3,5,8,15,25,40], include_lowest=True)

## remove unwanted layers e.g. certain event_types or certain actors
# list of all event types:
#    'Battles',
#    'Explosions/Remote violence',
#    'Protests',
#    'Riots',
#    'Strategic developments',
#    'Violence against civilians'

unwanted_layers = ['Protests','Riots','Strategic developments']
layers_needed = [i for i in geo_acled['event_type'].unique().tolist() if i not in unwanted_layers]
# in: layers_needed
# out: ['Explosions/Remote violence', 'Battles', 'Violence against civilians']

# limit dataframe to records where event_type are not in the unwanted list
geo_acled_limited = geo_acled.loc[geo_acled['event_type'].isin(layers_needed)]
geo_acled_limited = geo_acled_limited.reset_index(drop=True)

# CDS data Container for Bokeh
geo_acled_limited = geo_acled_limited.drop('geometry', axis=1)

source_acled = ColumnDataSource(geo_acled_limited)

In [10]:
# Initialize the plot (p) and give it a title
tile_provider_p = get_provider(STAMEN_TONER)
tile_provider_p1 = get_provider(STAMEN_TONER)
# tile_provider_p2 = get_provider(STAMEN_TONER)


#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.
scale = 5000
x=geo_acled_limited['mercator_lon']
y=geo_acled_limited['mercator_lat']

x_min=int(x.mean() - (scale*350))
x_max=int(x.mean() + (scale*350))
y_min=int(y.mean() - (scale*350))
y_max=int(y.mean() + (scale*350))

p = figure(plot_width=700, plot_height=500,
           x_range=(x_min, x_max),
           y_range=(y_min, y_max),
           x_axis_type="mercator", y_axis_type="mercator",
           match_aspect=True,
           tools='box_zoom,wheel_zoom,pan,box_select,lasso_select,reset,save',
           title="2022 Ukraine Conflict Data\nSource: www.acleddata.com​.",
           y_axis_location="left")

p1 = figure(plot_width=700, plot_height=500,
            x_range=p.x_range,
            y_range=p.y_range,
            x_axis_type="mercator", y_axis_type="mercator",
            match_aspect=True,
            tools='box_zoom,wheel_zoom,pan,box_select,lasso_select,reset,save',
            title="Data on transfer of children")

# p2 = figure(plot_width=700, plot_height=500,
#             x_range=p.x_range,
#             y_range=p.y_range,
#             x_axis_type="mercator", y_axis_type="mercator",
#             match_aspect=True,
#             tools='box_zoom,wheel_zoom,pan,box_select,lasso_select,reset,save',
#             title="Data on transfer of unidentified groups of children")

# p.grid.visible=True

p.add_tile(tile_provider_p)
# m.level='underlay'

# p1.grid.visible=True

p1.add_tile(tile_provider_p1)
# m1.level='underlay'

# p2.grid.visible=True

# p2.add_tile(tile_provider_p2)
# m1.level='underlay'

show(column(p,p1))

In [11]:
# https://discourse.bokeh.org/t/linking-daterangeslider-to-acled-data-on-bokeh-map/8848
# create factor colormap to map the event_type (categorical) to a palette - See https://docs.bokeh.org/en/latest/docs/reference/transform.html
layers_acled = layers_needed
layers_acled.sort()
layers_acled.reverse() # to reverse colorpalette

mapper_acled = factor_cmap(field_name='event_type',palette=OrRd3,factors=layers_acled)

# Create date range slider showing the ACLED data
#initializae date range slider here
#probably should be setting start end etc based on df['DateTime'].min() and max() but okay...
date_range_slider = DateRangeSlider(value=(geo_acled_limited['event_date'][len(geo_acled_limited['event_date'])-1], geo_acled_limited['event_date'][0]),
                                    start=geo_acled_limited['event_date'][len(geo_acled_limited['event_date'])-1], end=geo_acled_limited['event_date'][0]
                                   )
#create a customjs filter that returns the src indices the live between the slider values
filt = CustomJSFilter(args=dict(src=source_acled,date_range_slider=date_range_slider)
                      ,code='''
                      var minDate = date_range_slider.value[0]
                      var maxDate = date_range_slider.value[1]
                      var inds = []
                      //going through every entry DateTime, if it's in between the date range sliders, append to inds
                      for (var i = 0; i<src.data['event_date'].length; i++){
                              if (src.data['event_date'][i]>minDate && src.data['event_date'][i]<maxDate){
                                      inds.push(i)}
                              }
                      return inds'''
                      )                     

#one circle glyph/renderer for each event type, for the sake of having the legend click policy hide actually work
#https://stackoverflow.com/questions/64883994/all-data-being-remove-from-bokeh-plot-using-click-policy-hide 
# , with circle as marker, the mapper for the fill/line color
# and pointing to the fields in the datasource for the other args
#have each renderer use a view that uses two filters: the date filter and the groupfilter (for only that one specific event)
views = []
rends = []
for et in layers_acled:
    #create a groupfilter for the event
    gp_filt = GroupFilter(column_name='event_type',group=et)
    #create a cdsview that uses both the date filter and the group filter
    view = CDSView(source=source_acled,filters=[filt,gp_filt])
    rend = p.circle(x='mercator_lon', y='mercator_lat',fill_color=mapper_acled,line_color=mapper_acled,legend_label=et
                    ,hover_color='yellow',size='bin_fatalities',fill_alpha=0.4,source=source_acled,view=view)
    #need to keep track of all the views we create and all the rends we create
    views.append(view)
    rends.append(rend)
    
#tell all the views to update every time the slider values change
date_range_slider.js_on_change('value',CustomJS(args=dict(views=views)
                                                ,code='''
                                                for (var i=0;i<views.length;i++){ 
                                                        views[i].properties.filters.change.emit()
                                                        }'''))

#create one hover with all the renderers passed to it    
event_hover = HoverTool(tooltips=[
                            ("Date", "@event_date_ddmmyyyy"),
                            ("Country", "@country"),
                            ("Location", "@location"),
                            ("Event type", "@event_type"), 
                            ("Sub event type", "@sub_event_type"),
                            ("Actors", "@actor1, @actor2"),
                            ("Fatalities", "@fatalities"),   
                            # ("Notes", "@notes"),   
                            # ("Lat, Long", "@latitude, @longitude")
                            ],
                        mode='mouse',
                        point_policy='follow_mouse',
                        renderers=rends,
                        # formatters={'event_date': 'datetime'}
                       )

p.tools.append(event_hover)
# p.legend.location = "top_right"
# p.legend.click_policy="hide"

In [12]:
# Creating and adding layers fot all the children
# https://docs.bokeh.org/en/latest/docs/reference/colors.html#
# p1.multi_line(xs='lines_x', ys='lines_y', legend_label="movement", color="blue", source=CDS_source_children)
    
plot1 = p1.circle(x="mercator_lon_from", y="mercator_lat_from", size='bin_count_children', fill_alpha=0.5, legend_label="origin", 
         fill_color="lime", source=CDS_source_children)
p1.add_tools(HoverTool(renderers=[plot1], tooltips=[("Date of transfer", "@date_transfer_ddmmyyyy"), 
                                                    ("Moved from", "@from"), 
                                                    ("Moved to", "@to"), 
                                                    ("Number of children", "@count")]
                      )
            )

plot2 = p1.circle("mercator_lon_to", y="mercator_lat_to", size=10, fill_alpha=0.5, legend_label="destination", 
         fill_color="yellow", source=CDS_source_children)
p1.add_tools(HoverTool(renderers=[plot2], tooltips=[("Date of transfer", "@date_transfer_ddmmyyyy"), 
                                                    ("Moved from", "@from"), 
                                                    ("Moved to", "@to"), 
                                                    ("Number of children", "@count")]
                      )
            )

# Creating a table with data about the incidents of transfer of INDIVIDUAL children
# https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html#linked-properties

columns_datatable = [TableColumn(field='date_transfer', title='Date of transfer',
                       formatter=DateFormatter(format="%d-%m-%Y")),
                   TableColumn(field="from", title="Origin"),
                   TableColumn(field="to", title="Destination"),
                   TableColumn(field="count", title="Number of children",
                       formatter=NumberFormatter(format="0"))]

data_table1 = DataTable(source=CDS_source_children, columns=columns_datatable, editable=False, width=800,
                       index_position=-1, index_header="row index", index_width=60)

In [13]:
# # Creating and adding  layers from GROUPS of children
# p1.multi_line(lines_x_grp, lines_y_grp, legend_label="group movement", color="black")

# plot1 = p1.circle(x="mercator_lon_from", y="mercator_lat_from", size='bin_count_children', legend_label="group origin", 
#          fill_color="green", fill_alpha=0.5, source=CDS_source_grp)
# p1.add_tools(HoverTool(renderers=[plot1], tooltips=[('Date of transfer','@date_transfer_ddmmyyyy'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))

# plot2 = p1.circle(x="mercator_lon_to", y="mercator_lat_to", size=10, legend_label="group destination", 
#          fill_color="goldenrod", fill_alpha=0.5, source=CDS_source_grp)
# p1.add_tools(HoverTool(renderers=[plot2], tooltips=[('Date of transfer','@date_transfer_ddmmyyyy'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))


# # drop names of children ('Name of children','@name')

# # Creating a table with data about the incidents of transfer of INDIVIDUAL children
# # https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html#linked-properties

# columns = [TableColumn(field='date_transfer', title='Date of transfer',
#                        formatter=DateFormatter(format="%d-%m-%Y")),
#            TableColumn(field="from", title="Origin"),
#            TableColumn(field="to", title="Destination"),
#            TableColumn(field="count", title="Number of children",
#                        formatter=NumberFormatter(format="0"))]

# data_table2 = DataTable(source=CDS_source_grp, columns=columns, editable=False, width=800,
#                        index_position=-1, index_header="row index", index_width=60)

In [14]:
# Adding legend location  
p1.legend.location = "top_right"
p1.legend.click_policy="hide" # 'mute'

In [15]:
# grid = gridplot([[p, p1, p2], [date_range_slider, data_table1, data_table2]])

# layout = row(p1,data_table1)

show(data_table1)
# output_notebook()

ValueError: Out of range float values are not JSON compliant

In [ ]:
# #Create map
# # Initialize the plot (p) and give it a title
# tile_provider = get_provider(STAMEN_TONER)

# #The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.
# scale = 6000
# x=geo_acled['mercator_lon']
# y=geo_acled['mercator_lat']

# x_min=int(x.mean() - (scale*350))
# x_max=int(x.mean() + (scale*350))
# y_min=int(y.mean() - (scale*350))
# y_max=int(y.mean() + (scale*350))

# p = figure(plot_width=1500, plot_height=1000,
#                    x_range=(x_min, x_max),
#                    y_range=(y_min, y_max),
#                    x_axis_type="mercator", y_axis_type="mercator",
#                    match_aspect=True,
#                    tools='box_zoom,wheel_zoom,pan,reset,save',
#                    title="2022 Ukraine Conflict Data\nSource: www.acleddata.com​.")

# p.grid.visible=True

# m=p.add_tile(tile_provider)
# m.level='underlay'

# # Creating and adding layers from ACLED data with categories for each event type 
# events = geo_acled['event_type'].drop_duplicates().tolist()
# events.sort()

# for i in events:
#     temp_df = geo_acled.loc[(geo_acled['event_type'] == i)]
#     # temp_df = temp_df.drop('geometry', axis=1)
#     source_temp = ColumnDataSource(temp_df)
    
#     index = events.index(i)
    
#     circle = p.circle(x='mercator_lon', y='mercator_lat', source=source_temp, fill_color=Spectral6[index], line_color=Spectral6[index], 
#                       size='bin_fatalities', fill_alpha=0.5, legend_label=events[index], hover_color='yellow')
    
#     event_hover = HoverTool(tooltips=[
#                             ("Event type", "@event_type"), 
#                             ("Sub event type", "@sub_event_type"),
#                             ("Date", "@event_date"),
#                             ("Actors", "@actor1, @actor2"),
#                             ("Fatalities", "@fatalities"),   
#                             ("Notes", "@notes"),   
#                             ("Lat, Long", "@latitude, @longitude")
#                             ],
#                           mode='mouse',
#                           point_policy='follow_mouse',
#                           renderers=[circle])
    
#     event_hover.renderers.append(circle)
#     p.tools.append(event_hover)                                    


# # Creating and adding layers from individual children
# # https://docs.bokeh.org/en/latest/docs/reference/colors.html#
# p.multi_line(lines_x_ind, lines_y_ind, legend_label="movement", color="blue")
    
# plot1 = p.circle(x="mercator_lon_from", y="mercator_lat_from", size='bin_count_children', fill_alpha=0.5, legend_label="origin", 
#          fill_color="lime", source=CDS_source_ind)
# p.add_tools(HoverTool(renderers=[plot1], tooltips=[('Date of transfer','@date_transfer'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))

# plot2 = p.circle(x="mercator_lon_to", y="mercator_lat_to", size=10, fill_alpha=0.5, legend_label="destination", 
#          fill_color="yellow", source=CDS_source_ind)
# p.add_tools(HoverTool(renderers=[plot2], tooltips=[('Date of transfer','@date_transfer'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))

# # Creating and adding  layers from groups of children
# p.multi_line(lines_x_grp, lines_y_grp, legend_label="group movement", color="black")
# plot1 = p.circle(x="mercator_lon_from", y="mercator_lat_from", size='bin_count_children', legend_label="group origin", 
#          fill_color="green", fill_alpha=0.5, source=CDS_source_grp)
# p.add_tools(HoverTool(renderers=[plot1], tooltips=[('Date of transfer','@date_transfer'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))

# plot2 = p.circle(x="mercator_lon_to", y="mercator_lat_to", size=10, legend_label="group destination", 
#          fill_color="goldenrod", fill_alpha=0.5, source=CDS_source_grp)
# p.add_tools(HoverTool(renderers=[plot2], tooltips=[('Date of transfer','@date_transfer'), ("Moved from", "@from"), 
#                                                       ("Moved to", "@to"), ("Number of children", "@count")]))


# # drop names of children ('Name of children','@name')
# #Adding legend location    
# p.legend.location = "top_right"
# p.legend.click_policy="hide"

# # show(p)      

In [ ]:
# layout_with_widgets = column(row(
#                                 column(p, row(drop_scat1, drop_scat2)),
#                                 ))

# show(layout_with_widgets)                             

In [ ]:
# Give output filepath
outfp = r'UKR_ACLED/acled_map_test_grid.html'

# Save the plot by passing the plot -object and output path
save(obj=layout, filename=outfp)